<a href="https://colab.research.google.com/github/adityasodani03/advancepython/blob/main/Llama_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Nov 19 05:31:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
## GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 76.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

  Using cached llama_cpp_python-0.1.78.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp312-cp312-linux_x86_64.whl size=296684 sha256=1a5fd2e8267206e34e015785c26880d77e537cedb0ca17d1d3267ebcafdd2c07
  Stored in directory: /root/.cache/pip/wheels/79/ca/85/03c32b3b07d393042bf1c9f8d0349b96190a4f1c4f2b69f2f3
Successfully built llama-cpp-python
  Using cached numpy-1.23.4.tar.gz (10.7 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with

In [4]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [5]:
from huggingface_hub import hf_hub_download

In [6]:
from llama_cpp import Llama

In [7]:
#Download the model

In [8]:
model_path = hf_hub_download(repo_id=model_name_or_path , filename=model_basename)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [9]:
#### Loading the Model

In [10]:
## GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, ## CPU cores
    n_batch=512, ## Should be between 1 and n_ctx, consider the ammount of VRAM  in your GPU.
    n_gpu_layers=32 ## change this value based on your model and your GPU VRAM pool.
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [11]:
## See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [12]:
## 5. Create a Prompt Template

In [13]:
prompt = "What is NER IN NLP?"
prompt_template=f'''
SYSTEM: You are a helpful, respectful and honest assistant.Always answer as helpfully.
USER: {prompt}
ASSISTANT:
'''

In [14]:
## Generate Response

In [15]:
response=lcpp_llm(prompt=prompt_template,
                  max_tokens=256,
                  temperature=0.5,
                  top_p=0.95,
                  repeat_penalty=1.2,
                  top_k=150,
                  echo=True)


In [16]:
print(response)

{'id': 'cmpl-02f699de-8574-45f2-891b-6dfaa2caad9a', 'object': 'text_completion', 'created': 1763530728, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': '\nSYSTEM: You are a helpful, respectful and honest assistant.Always answer as helpfully.\nUSER: What is NER IN NLP?\nASSISTANT:\nNER (Named Entity Recognition) in Natural Language Processing (NLP) refers to the task of identifying and categorizing named entities in unstructured text into predefined categories such as person, organization, location, date, time, etc. It is an important subtask of Information Extraction and can be used for various applications like sentiment analysis, information retrieval, question answering, text summarization, and more.\n\nFor example, if we have a sentence "John Smith is the CEO of XYZ Corporation located in New York", NER would identify the named entities as "John 

In [17]:
print(response["choices"][0]["text"])


SYSTEM: You are a helpful, respectful and honest assistant.Always answer as helpfully.
USER: What is NER IN NLP?
ASSISTANT:
NER (Named Entity Recognition) in Natural Language Processing (NLP) refers to the task of identifying and categorizing named entities in unstructured text into predefined categories such as person, organization, location, date, time, etc. It is an important subtask of Information Extraction and can be used for various applications like sentiment analysis, information retrieval, question answering, text summarization, and more.

For example, if we have a sentence "John Smith is the CEO of XYZ Corporation located in New York", NER would identify the named entities as "John Smith" (person), "XYZ Corporation" (organization), and "New York" (location).

There are various techniques and approaches used for NER, including rule-based methods, machine learning algorithms like Support Vector Machines (SVM) and Convolutional Neural Networks (CNN), as well as hybrid models t